# How to call text-to-text DIAL applications

This notebook covers how to call text-to-text application via [DIAL API chat/completions](https://epam-rail.com/dial_api#/paths/~1openai~1deployments~1%7BDeployment%20Name%7D~1chat~1completions/post) call.

## Setup

Install the client libraries, which we will use to call the API.


In [4]:
! pip install "requests==2.31.0" "openai==1.9.0" "langchain-openai==0.0.3" >/dev/null

Run the docker compose file in a separate terminal to start the DIAL Core server locally.

```sh
(cd ../dial-docker-compose/application; docker compose up)
```

## Curl

The echo application deployment is called `echo`.

The local DIAL Core server URL is `http://localhost:8080`.

The OpenAI API version we are going to use is `2023-03-15-preview`.

Hence the echo application is accessible via the url:

```
http://localhost:8080/openai/deployments/echo/chat/completions?api-version=2023-03-15-preview
```

The corresponding curl command with a singe message in the request is:

In [23]:
! curl -X POST "http://localhost:8080/openai/deployments/echo/chat/completions?api-version=2023-03-15-preview" -H "Api-Key:dial_api_key" -H "Content-Type:application/json" -d '{"messages": [{"role": "user", "content": "Hello world!"}]}'

{"choices":[{"index":0,"finish_reason":"stop","message":{"role":"assistant","content":"Hello world!"}}],"usage":null,"id":"34c7bcf6-e94d-4f86-ba64-157ad9991bc5","created":1706007405,"object":"chat.completion"}

## Requests


Let's make an HTTP request from Python using `requests` library and make sure the return message matches the message in the request.

The arguments are identical to the curl command above.

### Non-streaming


In [24]:
import requests

response = requests.post(
    "http://localhost:8080/openai/deployments/echo/chat/completions?api-version=2023-03-15-preview",
    headers={"Api-Key": "dial_api_key"},
    json={"messages": [{"role": "user", "content": "Hello world!"}]},
)
body = response.json()
display(body)
completion = body["choices"][0]["message"]["content"]
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

{'choices': [{'index': 0,
   'finish_reason': 'stop',
   'message': {'role': 'assistant', 'content': 'Hello world!'}}],
 'usage': None,
 'id': '60b18019-2493-423c-b08b-b563ba1ebbab',
 'created': 1706007495,
 'object': 'chat.completion'}

Completion: 'Hello world!'


### Streaming

When streaming is enabled, the chat completion returns a sequence of message, each containing a chunk of a generated response.


In [25]:
response = requests.post(
    "http://localhost:8080/openai/deployments/echo/chat/completions?api-version=2023-03-15-preview",
    headers={"Api-Key": "dial_api_key"},
    json={"messages": [{"role": "user", "content": "Hello world!"}], "stream": True},
)
for chunk in response.iter_lines():
    print(chunk)

b'data: {"choices":[{"index":0,"finish_reason":null,"delta":{"role":"assistant"}}],"usage":null,"id":"50fde6fd-2927-46b8-9aae-750eed43ccb4","created":1706007518,"object":"chat.completion.chunk"}'
b''
b'data: {"choices":[{"index":0,"finish_reason":null,"delta":{"content":"Hello world!"}}],"usage":null,"id":"50fde6fd-2927-46b8-9aae-750eed43ccb4","created":1706007518,"object":"chat.completion.chunk"}'
b''
b'data: {"choices":[{"index":0,"finish_reason":"stop","delta":{}}],"usage":null,"id":"50fde6fd-2927-46b8-9aae-750eed43ccb4","created":1706007518,"object":"chat.completion.chunk"}'
b''
b'data: [DONE]'
b''


## OpenAI Python SDK

The DIAL deployment could be called using [OpenAI Python SDK](https://pypi.org/project/openai/) as well.


In [26]:
import openai

openai_client = openai.AzureOpenAI(
    azure_endpoint="http://localhost:8080",
    azure_deployment="echo",
    api_key="dial_api_key",
    api_version="2023-03-15-preview",
)

chat_completion = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello world!",
        }
    ],
    model="reverse-echo",
)
print(chat_completion)
completion = chat_completion.choices[0].message.content
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

ChatCompletion(id='babfc0ab-d375-4b45-9972-2305b933adef', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello world!', role='assistant', function_call=None, tool_calls=None))], created=1706007553, model=None, object='chat.completion', system_fingerprint=None, usage=None)
Completion: 'Hello world!'


## LangChain

Or via [LangChain](https://pypi.org/project/langchain-openai/) library


In [27]:
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="http://localhost:8080",
    azure_deployment="echo",
    api_key="dial_api_key",
    api_version="2023-03-15-preview",
)

output = llm.generate(messages=[[HumanMessage(content="Hello world!")]])
print(output)
completion = output.generations[0][0].text
print(f"Completion: {completion!r}")
assert completion == "Hello world!", "Unexpected completion"

generations=[[ChatGeneration(text='Hello world!', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Hello world!'))]] llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('c6a5c264-51ec-4bb0-aea8-bb96ea570e62'))]
Completion: 'Hello world!'
